In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
# Функции из 1-ого вебинара
# import os, sys

# module_path = os.path.abspath(os.path.join(os.pardir))
# if module_path not in sys.path:
#     sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k

In [2]:
# pip install implicit==0.6.0

In [3]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Подбор оптимальных гиперпараметров для ALS

- Попробуйте улучшить базовый вариант ALS, изменяя следующие параметры
  - regularization, iterations
  - factors
  - Вес (TF_IDF, BM25  взвешивание)
  
- Посчитайте метрики (Precision@5, MAP@5) для разных наборов гиперпараметров и выберете лучший набор

In [6]:
# your code
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/home/nal/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [55]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4,random_state=42)

model.fit(sparse_user_item,  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_id[999999]], 
                        recalculate_user=False)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 8.7 s, sys: 55.7 ms, total: 8.75 s
Wall time: 2.53 s


In [56]:
[id_to_itemid[rec] for rec in recs[0]]

[1106523, 1133018, 5569230, 1082185, 1068719]

In [57]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [58]:
def get_recommendations(user, model, sparse_user_item, N=5):
    res = [id_to_itemid[rec] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item[userid_to_id[user]],   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)[0]]
    return res

In [59]:
%%time
    
result['als'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

CPU times: user 11.5 s, sys: 228 ms, total: 11.7 s
Wall time: 3.27 s


0.1802154750244834

In [85]:
%%time

model = AlternatingLeastSquares(factors=10, 
                                regularization=0.001,
                                iterations=25, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).tocsr(),   # На вход item-user matrix
          show_progress=True)

  0%|          | 0/25 [00:00<?, ?it/s]

CPU times: user 11.6 s, sys: 23.6 ms, total: 11.7 s
Wall time: 3.6 s


In [86]:
%%time
    
result['my_als'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))
result.apply(lambda row: precision_at_k(row['my_als'], row['actual']), axis=1).mean()

CPU times: user 3.26 s, sys: 170 ms, total: 3.43 s
Wall time: 964 ms


0.19167482859940943

In [84]:
model.similar_items(1, N=5)

(array([   1, 2835, 2416, 4494, 4495], dtype=int32),
 array([0.99999994, 0.88380224, 0.8463278 , 0.84088826, 0.83707815],
       dtype=float32))

In [69]:
tfidf_user_item_matrix = tfidf_weight(user_item_matrix).tocsr() # Применяется к item-user матрице ! 

In [98]:
%%time

model = AlternatingLeastSquares(factors=70, 
                                regularization=0.1,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42)

model.fit(tfidf_user_item_matrix,  # На вход item-user matrix
          show_progress=True)

result['als_tfidf'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 11 s, sys: 202 ms, total: 11.2 s
Wall time: 3.11 s


0.24603330068559914

In [88]:
bm25_user_item_matrix = bm25_weight(user_item_matrix).tocsr() # Применяется к item-user матрице ! 

In [97]:
%%time

model = AlternatingLeastSquares(factors=70, 
                                regularization=0.1,
                                iterations=5,
                                calculate_training_loss=True, 
                                num_threads=4,
                                random_state=42) # K - кол-во билжайших соседей

model.fit(csr_matrix(bm25_user_item_matrix),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].map(lambda x: get_recommendations(x, model, sparse_user_item, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 11.3 s, sys: 236 ms, total: 11.5 s
Wall time: 3.22 s


0.2364348677766864

In [99]:
result

,user_id,actual,als,my_als,als_tfidf,als_bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1105488, 1033142, 5569374, 979707, 995242]","[862349, 1082185, 981760, 885290, 878996]","[1082185, 995242, 1100972, 1005186, 856942]","[5569374, 1082185, 995242, 979707, 901062]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 910032, 5569327, 1022003]","[1029743, 1082185, 8090521, 883404, 951590]","[951590, 826249, 1106523, 910032, 1092026]","[951590, 1106523, 1053690, 5569230, 1029743]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[866211, 1051516, 1007195, 878996, 904360]","[834484, 1037863, 1022254, 854852, 1024306]","[1082185, 1024306, 878996, 1127831, 965267]","[1082185, 1024306, 834484, 965267, 904360]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1058997, 1126899, 1082185, 938700, 893018]","[1082185, 6534178, 995242, 1029743, 981760]","[849843, 1082185, 6534178, 1003188, 938700]","[1003188, 1082185, 1122358, 1015247, 6944571]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[938700, 913785, 844179, 1068719, 1053690]","[1082185, 826249, 1098066, 981760, 1029743]","[1029743, 840361, 995242, 883404, 1082185]","[844165, 995242, 1068719, 1029743, 840361]"
...,...,...,...,...,...,...
2037,2496,[6534178],"[914190, 1013321, 1070702, 840361, 854852]","[844179, 826249, 1004906, 1098066, 1029743]","[844179, 840361, 1082185, 916122, 995785]","[6534178, 1133018, 1106523, 844179, 834484]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[5569230, 957951, 951590, 1051323, 908531]","[899624, 833025, 844179, 1004906, 923746]","[5569230, 951590, 826249, 845208, 899624]","[5569230, 899624, 1098066, 5569471, 981760]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[9526410, 1053690, 1070820, 1082185, 6534178]","[1082185, 6534178, 1029743, 995242, 981760]","[995242, 862349, 1070820, 6534178, 840361]","[1070820, 995242, 1053690, 6534178, 1029743]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[6534178, 929668, 1004906, 1098066, 826249]","[1029743, 1082185, 883404, 826249, 6534178]","[883404, 826249, 1029743, 6534178, 1098066]","[883404, 1053690, 1029743, 826249, 6534178]"


In [100]:
def ap_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list,bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(k):
        
        if flags[i]:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / k
    
    return result

In [101]:
result.apply(lambda row: ap_at_k(row['als_bm25'], row['actual']), axis=1).mean()

0.16050930460332946

In [102]:
result.apply(lambda row: ap_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

0.17010284035259549

In [104]:
result.apply(lambda row: ap_at_k(row['my_als'], row['actual']), axis=1).mean()

0.12606431603003518

In [105]:
result.apply(lambda row: ap_at_k(row['als'], row['actual']), axis=1).mean()

0.11038197845249717

Выводы: 
    Коэффициент Региляризации зависит от числа факторов и числа итераций - нужно подбирать гиперпараметры для получения лучшей модели.
    Взвешивание матрицы user_item_matrix с помощью bm25 и tfidf дают лучший результат, чем без него.
    Лучшая модель по метрикам - als_tfidf